<a href="https://colab.research.google.com/github/maxGrigorenko/RL/blob/prac6_Maksim_Grigorenko/practices/prac6/prac6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? В чистом policy-based алгоритме REINFORCE используется отдача $G_t$, полученная методом Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции — критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое [семейство](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf) различных алгоритмов. Рекомендую в качестве шпаргалки использовать упомянутый в тетрадке с REINFORCE [пост из блога Lilian Weng](https://lilianweng.github.io/posts/2018-04-08-policy-gradient/), посвященный наиболее популярным алгоритмам семейства актор-критиков

В данной тетрадке познакомимся с наиболее простым вариантом актор-критика, который так и называют Actor-Critic:

In [26]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [27]:
from collections import deque

import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.distributions import Categorical

%matplotlib inline

In [28]:
env = gym.make("CartPole-v1")
env.reset()

print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {env.observation_space.shape}')

env.observation_space=Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


(1 балл)

In [29]:
def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x)
    return x

def symlog(x):
    """Compute symlog values for a vector `x`. It's an inverse operation for symexp."""
    return torch.sign(x) * torch.log(torch.abs(x) + 1)

def symexp(x):
    """Compute symexp values for a vector `x`. It's an inverse operation for symlog."""
    return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)


class SymExpModule(nn.Module):
    def forward(self, x):
        return symexp(x)

def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    if np.random.random() < epsilon:
        action = np.random.randint(len(Q_s))
    else:
        action = np.argmax(Q_s)

    action = int(action)
    return action

def sample_batch(replay_buffer, n_samples):
    # sample randomly `n_samples` samples from replay buffer
    # and split an array of samples into arrays: states, actions, rewards, next_actions, terminateds
    batch = np.random.choice(len(replay_buffer), size=n_samples, replace=False)

    states = [replay_buffer[i][0] for i in batch]
    actions = [replay_buffer[i][1] for i in batch]
    rewards = [replay_buffer[i][2] for i in batch]
    next_states = [replay_buffer[i][3] for i in batch]
    terminateds = [replay_buffer[i][4] for i in batch]

    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

## Shared-body Actor-Critic

Актор и критик могут обучаться в разных режимах — актор только on-policy (шаг обучения на текущей собранной подтраектории), а критик on-policy или off-policy (шаг обучения на текущей подтраектории или на батче из replay buffer). Это с одной стороны привносит гибкость в обучение, с другой — усложняет его.

Если актор и критик оба обучаются on-policy, то имеет смысл объединить их сетки в одну и делать общий шаг обратного распространения ошибки. Однако, если они обучаются в разных режимах (и с разной частотой обновления), то велика вероятность, что их шаги обучения могут начать конфликтовать в случае общего тела — для такого варианта намного предпочтительнее разделить их на разные подсети (либо аккуратно настраивать гиперпарметры, чтобы стабилизировать обучение). В целом, рекомендуется использовать общий энкодер наблюдений, а далее как можно скорее разделять головы.

Сделаем реализацию актор-критика с общим телом и с on-policy вариантом обучения.

In [30]:
class ActorBatch:
    def __init__(self):
        self.states = []
        self.actions = []
        self.logprobs = []
        self.rewards = []
        self.next_states = []
        self.terminateds = []

    def append(self, state, action, log_prob, reward, next_state, terminated):
        self.states.append(state)
        self.actions.append(action)
        self.logprobs.append(log_prob)
        self.rewards.append(reward)
        self.next_states.append(next_state)
        self.terminateds.append(terminated)

    def clear(self):
        self.states.clear()
        self.actions.clear()
        self.logprobs.clear()
        self.rewards.clear()
        self.next_states.clear()
        self.terminateds.clear()


(3 балла)

In [31]:
class ActorModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim

        self.net = nn.Sequential(*layers)
        self.head = nn.Sequential(
            nn.Linear(prev_dim, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, state):
        features = self.net(state)
        action_probs = self.head(features)
        return action_probs

    def get_action(self, state):
        action_probs = self.forward(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action, log_prob


In [32]:
class CriticModel(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim

        self.net = nn.Sequential(*layers)
        self.head = nn.Linear(prev_dim, 1)

    def forward(self, state):
        features = self.net(state)
        state_value = self.head(features).squeeze(-1)
        return state_value

(6 баллов)

In [35]:
class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_dims, lr, gamma, critic_rb_size):
        self.lr = lr
        self.gamma = gamma

        self.actor = ActorModel(state_dim[0], hidden_dims, action_dim)
        self.critic = CriticModel(state_dim[0], hidden_dims)

        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=lr)

        self.actor_batch = ActorBatch()
        self.critic_rb = deque(maxlen=critic_rb_size)

    def act(self, state):
        state_tensor = to_tensor(state).unsqueeze(0)
        action, log_prob = self.actor.get_action(state_tensor)
        self.actor_batch.append(state, action.item(), log_prob, 0, state, False)
        return action.item()

    def append_to_replay_buffer(self, s, a, r, next_s, terminated):
        if self.actor_batch.states:
            self.actor_batch.rewards[-1] = r
            self.actor_batch.next_states[-1] = next_s
            self.actor_batch.terminateds[-1] = terminated
        self.critic_rb.append((s, a, r, next_s, terminated))

    def evaluate(self, state):
        return self.critic(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor):
        if len(self.actor_batch.states) < rollout_size:
            return

        critic_updated = False
        if len(self.critic_rb) >= critic_batch_size:
            critic_loss = self.update_critic(critic_batch_size, critic_updates_per_actor)
            if critic_loss is not None:
                self.critic_optimizer.zero_grad()
                critic_loss.backward()
                self.critic_optimizer.step()
                critic_updated = True

        if critic_updated:
            actor_loss = self.update_actor()
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
        self.actor_batch.clear()

    def update_actor(self):
        states = to_tensor(np.array(self.actor_batch.states))
        logprobs = torch.stack(self.actor_batch.logprobs)
        rewards = to_tensor(np.array(self.actor_batch.rewards))
        next_states = to_tensor(np.array(self.actor_batch.next_states))
        terminateds = to_tensor(np.array(self.actor_batch.terminateds), bool)

        with torch.no_grad():
            current_values = self.critic(states)
            next_values = self.critic(next_states)
            targets = rewards + self.gamma * next_values * (~terminateds)
            advantages = targets - current_values

        loss = -(logprobs * advantages).mean()
        return loss

    def update_critic(self, batch_size, n_updates=1):
        total_loss = 0
        update_count = 0

        for _ in range(n_updates):
            if len(self.critic_rb) < batch_size:
                continue

            states, actions, rewards, next_states, terminateds = sample_batch(self.critic_rb, batch_size)
            loss = self.compute_td_loss(states, actions, rewards, next_states, terminateds)
            total_loss += loss
            update_count += 1

        if update_count == 0:
            return None

        return total_loss / update_count

    def compute_td_loss(self, states, actions, rewards, next_states, terminateds):
        s = to_tensor(states)
        r = to_tensor(rewards)
        s_next = to_tensor(next_states)
        term = to_tensor(terminateds, bool)
        current_values = self.critic(s)

        with torch.no_grad():
            next_values = self.critic(s_next)

        targets = r + self.gamma * next_values * (~term)
        td_error = current_values - targets
        loss = torch.mean(td_error ** 2)
        return loss

In [36]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=4,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape,
        action_dim=env.action_space.n,
        hidden_dims=hidden_dims,
        lr=lr,
        gamma=0.99,
        critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        agent.update(train_schedule, batch_size, critic_updates_per_actor)

        if global_step % eval_schedule == 0:
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                print(f'{global_step=} | {avg_return=:.3f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(eval_schedule=2000, total_max_steps=100_000)

global_step=2008 | avg_return=11.000
global_step=4005 | avg_return=12.500
global_step=6020 | avg_return=18.667
global_step=8102 | avg_return=40.750
global_step=10036 | avg_return=60.400
global_step=12043 | avg_return=64.000
global_step=14065 | avg_return=87.857
global_step=16089 | avg_return=90.250
global_step=18052 | avg_return=98.333
global_step=20046 | avg_return=101.500
global_step=22201 | avg_return=127.200
global_step=24097 | avg_return=139.100
global_step=26203 | avg_return=157.300
global_step=28198 | avg_return=174.700
global_step=30068 | avg_return=184.600
global_step=32138 | avg_return=226.400
Решено!
